In [2]:
import pandas as pd
import numpy as npy
from datetime import date, datetime
import random
#the input is the csv files fight_hist and fighter_stats which we scrape from ufcstats.com
#careful, might say file does not exist if its saved to cloud (not on local device)
ufcfightscrap=pd.read_csv('fight_hist.csv',sep=',')
ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')

In [1]:
#this cell contains all functions defined for building columns in ufcfightscrap
#converts from '%B %d, %Y' (i.e. August 22, 2020) to date (i.e. 2020-08-22)
def convert_to_datetime(day1):
    return datetime.strptime(day1, '%B %d, %Y').date()
convert_to_datetime_vect= npy.vectorize(convert_to_datetime)

#this age function is written in such a stupid way
def age(birthDate,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    if birthDate=='--':
        aa='unknown'
    elif type(birthDate)==str and not type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif type(birthDate)==str and type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif not type(birthDate)==str and type(day)==str:
        bd=birthDate
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    else:
        bd=birthDate
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    return float(aa)

age_vect= npy.vectorize(age)

def fighter_age(fighter,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            dob=datetime.strptime(ufcfighterscrap['dob'][i], '%b %d, %Y').strftime('%B %d, %Y')
            a=age(dob,day,form1,form2)
            break
    return float(a)

def wins_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
    return float(summ)

def losses_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
    return float(summ)

#functions for height and reach
def fighter_height(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['height'][i]
            break
    if a=='--':
        b='unknown'
    elif a[4]=='"':
        b=int(a[0])*30.48+int(a[3])*2.54
    else:
        b=int(a[0])*30.48+int(a[3]+a[4])*2.54
    try:
        return float(b)
    except:
        return b

def fighter_reach(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['reach'][i]
            break
    if a=='--':
        b='unknown'
    else:
        b=int(a[0]+a[1])*2.54
    try:
        return float(b)
    except:
        return b

wins_before_vect= npy.vectorize(wins_before)
losses_before_vect= npy.vectorize(losses_before)
fighter_height_vect= npy.vectorize(fighter_height)
fighter_reach_vect= npy.vectorize(fighter_reach)

#day1 should be input in the form '%B %d, %Y' i.e. 'August 20, 1962'
#conversions can be made via day=datetime.strptime(ufcfightsML_known_df['date'][i], '%B %d, %Y').strftime('%b %d, %Y')
def time_diff(day1,day2=date.today()):
    if day2==date.today():
        answer=(day2-datetime.strptime(day, '%B %d, %Y')).days
    else:
        answer=(datetime.strptime(day2, '%B %d, %Y')-datetime.strptime(day1, '%B %d, %Y')).days
    return answer

#we now vectorize this to use in pandas/numpy
time_diff_vect= npy.vectorize(time_diff)

#can make a single function to do all of these... actually maybe the count function would even work as is

def L5Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return float(summ)

def ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return float(summ)

def L5Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return float(summ)

def sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1) and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return float(summ)

def L5Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L5Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return float(summ)

def L2Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

def L2Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return float(summ)

#vectorize all of these functions
L5Y_wins_vect= npy.vectorize(L5Y_wins)
L5Y_losses_vect= npy.vectorize(L5Y_losses)
L2Y_wins_vect= npy.vectorize(L2Y_wins)
L2Y_losses_vect= npy.vectorize(L2Y_losses)
ko_wins_vect= npy.vectorize(ko_wins)
ko_losses_vect= npy.vectorize(ko_losses)
L5Y_ko_wins_vect= npy.vectorize(L5Y_ko_wins)
L5Y_ko_losses_vect= npy.vectorize(L5Y_ko_losses)
L2Y_ko_wins_vect= npy.vectorize(L2Y_ko_wins)
L2Y_ko_losses_vect= npy.vectorize(L2Y_ko_losses)
sub_wins_vect= npy.vectorize(sub_wins)
sub_losses_vect= npy.vectorize(sub_losses)
L5Y_sub_wins_vect= npy.vectorize(L5Y_sub_wins)
L5Y_sub_losses_vect= npy.vectorize(L5Y_sub_losses)
L2Y_sub_wins_vect= npy.vectorize(L2Y_sub_wins)
L2Y_sub_losses_vect= npy.vectorize(L2Y_sub_losses)

#for columns like fighter_rec which contains the information for the opponent as well, we use the following
def opponent_column(stat):
    col=dict()
    for i in range(len(ufcfightscrap['fighter'])):
        if i%2==0:
            col[i]=ufcfightscrap[stat][i+1]
        else:
            col[i]=ufcfightscrap[stat][i-1]
    statdict={'stat':col}
    return pd.DataFrame (statdict, columns = ['stat'])

#enter date unabbreviated 'July 4, 2019'
#here the average gives avg per fight. Later in avg_count we change to average per time spent in octagon
def count(stat, guy,inf_abs, total_L5Y_L2Y_avg, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if total_L5Y_L2Y_avg=='total' or total_L5Y_L2Y_avg=='avg':
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0]
    elif total_L5Y_L2Y_avg=='L2Y':
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<730]
    else:
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<1825]
    if inf_abs=='inf':
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    else:
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy]
    good_indices=[i for i in good_indices_1 if i in good_indices_2]
    if total_L5Y_L2Y_avg!='avg':
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
    else:
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
        day1=convert_date_to_abbrev(day1)
        number_fights=wins_before(guy,day1)+losses_before(guy,day1)
        summ=summ/float(number_fights)
    return summ
        
#note a better average is per time not per fight. We will do each stat as an average per one minutes

#enter time in the form 'August 24, 2018'
def time_in_octagon(guy,day1=date.today().strftime('%B %d, %Y')):
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['fighter'][i]==guy] 
    for i in good_indices:
        if ufcfightscrap['time'][i][2]==':':
            summ=int(ufcfightscrap['time'][i][0:2])+int(ufcfightscrap['time'][i][3:])/60.0
        else:
            summ+=5*(ufcfightscrap['round'][i]-1)+int(ufcfightscrap['time'][i][0])+int(ufcfightscrap['time'][i][2:])/60.0
    return summ

#enter date unabbreviated 'July 4, 2019'
#gives takedowns per minute
def avg_count(stat, guy,inf_abs, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if inf_abs=='inf':
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    else:
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    for i in good_indices:
        summ+=ufcfightscrap[stat][i]
    t= time_in_octagon(guy,day1)
    if t==0:
        summ=0
    else:
        summ=summ/t
    return summ

#vectorize these functions
count_vect= npy.vectorize(count)
avg_count_vect= npy.vectorize(avg_count)
    


def stance(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['stance'][i]
            break
    if a=='Orthodox':
        return 0
    elif a=='Switch':
        return 1
    elif a=='Southpaw':
        return 2
    elif a=='Open Stance':
        return 3
    elif a=='Sideways':
        return 4
    else:
        return 5
    
stance_vect= npy.vectorize(stance)

NameError: name 'npy' is not defined

In [ ]:
#adding a datetime column for ufcfightscrap
#ufcfightscrap['datetime']=convert_to_datetime_vect(ufcfightscrap['date'])
#run time for each is 20 min each
ufcfightscrap['fighter_wins']=wins_before_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_losses']=losses_before_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_age']=fighter_age_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_height']=fighter_height_vect(ufcfightscrap['fighter'])
ufcfightscrap['fighter_reach']=fighter_reach_vect(ufcfightscrap['fighter'])

In [ ]:
#making columns for fighter_L5Y_wins losses etc
ufcfightscrap['fighter_L5Y_wins']=L5Y_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L5Y_losses']=L5Y_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_wins']=L2Y_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_losses']=L2Y_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_ko_wins']=ko_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_ko_losses']=ko_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L5Y_ko_wins']=L5Y_ko_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L5Y_ko_losses']=L5Y_ko_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_ko_wins']=L2Y_ko_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_ko_losses']=L2Y_ko_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_sub_wins']=sub_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_sub_losses']=sub_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L5Y_sub_wins']=L5Y_sub_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L5Y_sub_losses']=L5Y_sub_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_sub_wins']=L2Y_sub_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_sub_losses']=L2Y_sub_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])

In [ ]:
#forming columns for fighter_inf_knockdowns_avg etc...
#expect this to take about 2.5 hours per cell
ufcfightscrap['fighter_inf_knockdowns_avg']=avg_count_vect('knockdowns',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_pass_avg']=avg_count_vect('pass',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_reversals_avg']=avg_count_vect('reversals',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_sub_attempts_avg']=avg_count_vect('sub_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_takedowns_landed_avg']=avg_count_vect('takedowns_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])
ufcfightscrap['fighter_inf_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date'])


In [ ]:
#forming columns for fighter_abs_knockdowns_avg etc...
#expect this to take about 2.5 hours per cell
ufcfightscrap['fighter_abs_knockdowns_avg']=avg_count_vect('knockdowns',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_pass_avg']=avg_count_vect('pass',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_reversals_avg']=avg_count_vect('reversals',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_sub_attempts_avg']=avg_count_vect('sub_attempts',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_takedowns_landed_avg']=avg_count_vect('takedowns_landed',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])
ufcfightscrap['fighter_abs_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',ufcfightscrap['fighter'],'abs',ufcfightscrap['date'])

In [ ]:
#runs immediately
ufcfightscrap['opponent_wins']=opponent_column('fighter_wins')
ufcfightscrap['opponent_losses']=opponent_column('fighter_losses')
ufcfightscrap['opponent_age']=opponent_column('fighter_age')
ufcfightscrap['opponent_height']=opponent_column('fighter_height')
ufcfightscrap['opponent_reach']=opponent_column('fighter_reach')
#making columns for opponent_L5Y_wins losses etc
ufcfightscrap['opponent_L5Y_wins']=opponent_column('fighter_L5Y_wins')
ufcfightscrap['opponent_L5Y_losses']=opponent_column('fighter_L5Y_losses')
ufcfightscrap['opponent_L2Y_wins']=opponent_column('fighter_L2Y_wins')
ufcfightscrap['opponent_L2Y_losses']=opponent_column('fighter_L2Y_losses')
ufcfightscrap['opponent_ko_wins']=opponent_column('fighter_ko_wins')
ufcfightscrap['opponent_ko_losses']=opponent_column('fighter_ko_losses')
ufcfightscrap['opponent_L5Y_ko_wins']=opponent_column('fighter_L5Y_ko_wins')
ufcfightscrap['opponent_L5Y_ko_losses']=opponent_column('fighter_L5Y_ko_losses')
ufcfightscrap['opponent_L2Y_ko_wins']=opponent_column('fighter_L2Y_ko_wins')
ufcfightscrap['opponent_L2Y_ko_losses']=opponent_column('fighter_L2Y_ko_losses')
ufcfightscrap['opponent_sub_wins']=opponent_column('fighter_sub_wins')
ufcfightscrap['opponent_sub_losses']=opponent_column('fighter_sub_losses')
ufcfightscrap['opponent_L5Y_sub_wins']=opponent_column('fighter_L5Y_sub_wins')
ufcfightscrap['opponent_L5Y_sub_losses']=opponent_column('fighter_L5Y_sub_losses')
ufcfightscrap['opponent_L2Y_sub_wins']=opponent_column('fighter_L2Y_sub_wins')
ufcfightscrap['opponent_L2Y_sub_losses']=opponent_column('fighter_L2Y_sub_losses')

In [ ]:
#forming columns for opponent_inf_knockdowns_avg etc...
#expect this to take about 2.5 hours per cell
ufcfightscrap['opponent_inf_knockdowns_avg']=avg_count_vect('knockdowns',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_pass_avg']=avg_count_vect('pass',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_reversals_avg']=avg_count_vect('reversals',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_sub_attempts_avg']=avg_count_vect('sub_attempts',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_takedowns_landed_avg']=avg_count_vect('takedowns_landed',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])
ufcfightscrap['opponent_inf_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',ufcfightscrap['opponent'],'inf',ufcfightscrap['date'])

In [ ]:
#forming columns for opponent_abs_knockdowns_avg etc...
#expect this to take about 2.5 hours per cell
ufcfightscrap['opponent_abs_knockdowns_avg']=avg_count_vect('knockdowns',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_pass_avg']=avg_count_vect('pass',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_reversals_avg']=avg_count_vect('reversals',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_sub_attempts_avg']=avg_count_vect('sub_attempts',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_takedowns_landed_avg']=avg_count_vect('takedowns_landed',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_leg_strikes_landed_avg']=avg_count_vect('leg_strikes_landed',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_leg_strikes_attempts_avg']=avg_count_vect('leg_strikes_attempts',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_distance_strikes_landed_avg']=avg_count_vect('distance_strikes_landed',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_distance_strikes_attempts_avg']=avg_count_vect('distance_strikes_attempts',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_clinch_strikes_landed_avg']=avg_count_vect('clinch_strikes_landed',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_clinch_strikes_attempts_avg']=avg_count_vect('clinch_strikes_attempts',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_ground_strikes_landed_avg']=avg_count_vect('ground_strikes_landed',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])
ufcfightscrap['opponent_abs_ground_strikes_attempts_avg']=avg_count_vect('ground_strikes_attempts',ufcfightscrap['opponent'],'abs',ufcfightscrap['date'])

In [140]:
ufcfightscrap['fighter_stance']=stance_vect(ufcfightscrap['fighter'])
ufcfightscrap['opponent_stance']=stance_vect(ufcfightscrap['opponent'])

In [ ]:
ufcfightscrap['1-fight_math'] = fight_math_diff_vect(ufcfightscrap['fighter'], ufcfightscrap['opponent'], ufcfightscrap['date'],1)
ufcfightscrap['6-fight_math'] = fight_math_diff_vect(ufcfightscrap['fighter'], ufcfightscrap['opponent'], ufcfightscrap['date'],6)

ufcfightscrap['4-fighter_score_diff'] = fighter_score_diff_vect(ufcfightscrap['fighter'], ufcfightscrap['opponent'], ufcfightscrap['date'],4)
ufcfightscrap['9-fighter_score_diff'] = fighter_score_diff_vect(ufcfightscrap['fighter'], ufcfightscrap['opponent'], ufcfightscrap['date'],9)
ufcfightscrap['15-fighter_score_diff'] = fighter_score_diff_vect(ufcfightscrap['fighter'], ufcfightscrap['opponent'], ufcfightscrap['date'],15)



In [143]:
ufcfightscrap

,date,division,event_url,fight_url,fighter,fighter_url,method,opponent,opponent_url,result,...,opponent_abs_leg_strikes_landed_avg,opponent_abs_leg_strikes_attempts_avg,opponent_abs_distance_strikes_landed_avg,opponent_abs_distance_strikes_attempts_avg,opponent_abs_clinch_strikes_landed_avg,opponent_abs_clinch_strikes_attempts_avg,opponent_abs_ground_strikes_landed_avg,opponent_abs_ground_strikes_attempts_avg,fighter_stance,opponent_stance
0,"August 15, 2020",Heavyweight,http://ufcstats.com/event-details/bda04c573563...,http://ufcstats.com/fight-details/3bdacc82209b...,Stipe Miocic,http://ufcstats.com/fighter-details/d28dee5c70...,U-DEC,Daniel Cormier,http://ufcstats.com/fighter-details/d967f0128c...,W,...,0.548196,0.740064,2.571037,5.849246,0.657835,1.003198,0.131567,0.180905,0,0
1,"August 15, 2020",Heavyweight,http://ufcstats.com/event-details/bda04c573563...,http://ufcstats.com/fight-details/3bdacc82209b...,Daniel Cormier,http://ufcstats.com/fighter-details/d967f0128c...,U-DEC,Stipe Miocic,http://ufcstats.com/fighter-details/d28dee5c70...,L,...,0.422659,0.534194,3.164074,7.772234,0.305254,0.487232,0.217200,0.287643,0,0
2,"August 15, 2020",Bantamweight,http://ufcstats.com/event-details/bda04c573563...,http://ufcstats.com/fight-details/7200343d2e95...,Marlon Vera,http://ufcstats.com/fighter-details/7c7332319c...,KO/TKO,Sean O'Malley,http://ufcstats.com/fighter-details/b50a426a33...,W,...,0.618861,1.001965,3.035363,9.990177,0.176817,0.530452,0.235756,0.265226,1,1
3,"August 15, 2020",Bantamweight,http://ufcstats.com/event-details/bda04c573563...,http://ufcstats.com/fight-details/7200343d2e95...,Sean O'Malley,http://ufcstats.com/fighter-details/b50a426a33...,KO/TKO,Marlon Vera,http://ufcstats.com/fighter-details/7c7332319c...,L,...,0.680183,0.981033,2.858077,6.906475,0.516678,0.778286,0.732505,1.013734,1,1
4,"August 15, 2020",Heavyweight,http://ufcstats.com/event-details/bda04c573563...,http://ufcstats.com/fight-details/dc24c4759651...,Jairzinho Rozenstruik,http://ufcstats.com/fighter-details/2cd428e960...,KO/TKO,Junior Dos Santos,http://ufcstats.com/fighter-details/63def5a566...,W,...,0.507225,0.606936,2.167630,6.099711,0.671965,1.005780,0.290462,0.411850,0,0
5,"August 15, 2020",Heavyweight,http://ufcstats.com/event-details/bda04c573563...,http://ufcstats.com/fight-details/dc24c4759651...,Junior Dos Santos,http://ufcstats.com/fighter-details/63def5a566...,KO/TKO,Jairzinho Rozenstruik,http://ufcstats.com/fighter-details/2cd428e960...,L,...,0.786164,0.849057,2.138365,3.679245,0.314465,0.345912,0.754717,0.849057,0,0
6,"August 15, 2020",Featherweight,http://ufcstats.com/event-details/bda04c573563...,http://ufcstats.com/fight-details/0d9e9127c628...,Daniel Pineda,http://ufcstats.com/fighter-details/361d49960a...,KO/TKO,Herbert Burns,http://ufcstats.com/fighter-details/4f14040300...,W,...,0.000000,0.000000,1.234568,2.962963,0.493827,0.740741,0.000000,0.000000,0,0
7,"August 15, 2020",Featherweight,http://ufcstats.com/event-details/bda04c573563...,http://ufcstats.com/fight-details/0d9e9127c628...,Herbert Burns,http://ufcstats.com/fighter-details/4f14040300...,KO/TKO,Daniel Pineda,http://ufcstats.com/fighter-details/361d49960a...,L,...,0.194301,0.233161,1.690415,3.516839,0.485751,0.641192,0.330311,0.641192,0,0
8,"August 15, 2020",Bantamweight,http://ufcstats.com/event-details/bda04c573563...,http://ufcstats.com/fight-details/57939328c121...,Merab Dvalishvili,http://ufcstats.com/fighter-details/c03520b5c8...,U-DEC,John Dodson,http://ufcstats.com/fighter-details/99506df1af...,W,...,0.600231,0.900346,2.839554,8.306272,0.706426,0.969604,0.046172,0.055406,0,0
9,"August 15, 2020",Bantamweight,http://ufcstats.com/event-details/bda04c573563...,http://ufcstats.com/fight-details/57939328c121...,John Dodson,http://ufcstats.com/fighter-details/99506df1af...,U-DEC,Merab Dvalishvili,http://ufcstats.com/fighter-details/c03520b5c8...,L,...,0.355556,0.444444,1.355556,4.133333,0.644444,0.977778,0.122222,0.155556,0,0


In [16]:
#here is the list of all stats available, does not include names or result
good_statistics=[u'fighter_wins', u'fighter_losses', u'fighter_age',u'fighter_height',
                    u'fighter_reach', u'fighter_L5Y_wins',u'fighter_L5Y_losses', u'fighter_L2Y_wins', u'fighter_L2Y_losses',
                    u'fighter_ko_wins',u'fighter_ko_losses',u'fighter_L5Y_ko_wins',u'fighter_L5Y_ko_losses',u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',u'fighter_sub_wins',u'fighter_sub_losses',u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', u'fighter_L2Y_sub_wins', u'fighter_L2Y_sub_losses',
                    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    u'opponent_wins', u'opponent_losses', u'opponent_age',  u'opponent_height',
                    u'opponent_reach',   u'opponent_L5Y_wins', u'opponent_L5Y_losses', u'opponent_L2Y_wins', u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', u'opponent_ko_losses', u'opponent_L5Y_ko_wins', u'opponent_L5Y_ko_losses', u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', u'opponent_sub_wins', u'opponent_sub_losses',u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', u'opponent_L2Y_sub_wins', u'opponent_L2Y_sub_losses',
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg',
                    u'fighter_stance',
                    u'opponent_stance',
                 'fighter_age_diff',
                '1-fight_math',
                '6-fight_math',
                '4-fighter_score_diff',
                '9-fighter_score_diff',
                 '15-fighter_score_diff',
                ]




In [145]:
#we worked hard to build this, lets save it
ufcfightscrap.to_csv('ufc_fights_crap.csv', index = False)

In [146]:
#saving a hard copy to never overwrite
ufcfightscrap.to_csv('ufc_fights_crap_Sept_7_2020_DO_NOT_OVERWRITE.csv', index = False)

In [155]:
#list containing all columns of any interest
relevant_list=['date','division','fighter','opponent','result','method']
relevant_list.extend(good_statistics)

In [156]:
#creates a clean file with only columns which are relevant to predicting
ufc_fights=ufcfightscrap[relevant_list]

In [157]:
#lets randomly remove one of every two copied fights
random_indices=[]
for i in range(0,len(ufc_fights['fighter_wins']),2):
    random_indices.append(random.choice([i,i+1]))
    
ufc_fights=ufc_fights.drop(random_indices)

In [161]:
#we worked hard to build this, lets save it
ufc_fights.to_csv('ufc_fights.csv', index = False)

In [158]:
ufc_fights

,date,division,fighter,opponent,result,method,fighter_wins,fighter_losses,fighter_age,fighter_height,...,opponent_abs_leg_strikes_landed_avg,opponent_abs_leg_strikes_attempts_avg,opponent_abs_distance_strikes_landed_avg,opponent_abs_distance_strikes_attempts_avg,opponent_abs_clinch_strikes_landed_avg,opponent_abs_clinch_strikes_attempts_avg,opponent_abs_ground_strikes_landed_avg,opponent_abs_ground_strikes_attempts_avg,fighter_stance,opponent_stance
1,"August 15, 2020",Heavyweight,Daniel Cormier,Stipe Miocic,L,U-DEC,11,2,41,180.34,...,0.422659,0.534194,3.164074,7.772234,0.305254,0.487232,0.217200,0.287643,0,0
3,"August 15, 2020",Bantamweight,Sean O'Malley,Marlon Vera,L,KO/TKO,4,0,25,180.34,...,0.680183,0.981033,2.858077,6.906475,0.516678,0.778286,0.732505,1.013734,1,1
5,"August 15, 2020",Heavyweight,Junior Dos Santos,Jairzinho Rozenstruik,L,KO/TKO,15,6,36,193.04,...,0.786164,0.849057,2.138365,3.679245,0.314465,0.345912,0.754717,0.849057,0,0
7,"August 15, 2020",Featherweight,Herbert Burns,Daniel Pineda,L,KO/TKO,2,0,32,175.26,...,0.194301,0.233161,1.690415,3.516839,0.485751,0.641192,0.330311,0.641192,0,0
8,"August 15, 2020",Bantamweight,Merab Dvalishvili,John Dodson,W,U-DEC,4,2,29,167.64,...,0.600231,0.900346,2.839554,8.306272,0.706426,0.969604,0.046172,0.055406,0,0
11,"August 15, 2020",Lightweight,Jim Miller,Vinc Pichel,L,U-DEC,21,13,36,172.72,...,0.293464,0.346821,1.680747,4.682081,0.453535,0.586928,0.413517,0.560249,2,0
13,"August 15, 2020",Women's Strawweight,Felice Herrig,Virna Jandiroba,L,SUB,5,3,35,162.56,...,0.047022,0.047022,0.893417,2.680251,0.470219,0.705329,0.235110,0.423197,0,0
14,"August 15, 2020",Featherweight,Danny Chavez,TJ Brown,W,U-DEC,0,0,33,172.72,...,0.000000,0.000000,1.042471,1.969112,0.347490,0.347490,0.347490,0.347490,0,1
17,"August 15, 2020",Women's Strawweight,Ashley Yoder,Livinha Souza,L,U-DEC,2,4,32,170.18,...,0.350877,0.606061,2.456140,7.591707,0.701754,1.180223,0.287081,0.350877,2,0
18,"August 15, 2020",Heavyweight,Chris Daukaus,Parker Porter,W,KO/TKO,0,0,30,190.5,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0


In [160]:
#saving a hard copy to never overwrite
ufc_fights.to_csv('ufc_fights_Sept_7_2020_DO_NOT_OVERWRITE.csv', index = False)

In [ ]:
#The files we end up with are ufc_fights_crap which contains 2 copies of each fight, 
#statistics for what happened in that fight, and statistics for what happened to each fighter in every previous fight
#We also end up with ufc_fights which is a cleaned version, containing only relevant statistics for prediction

In [6]:
#fixing a mistake
ufcfightscrap=pd.read_csv('ufc_fights_crap.csv',sep=',')

/opt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (36,37,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
#correcting mistakes
ufcfightscrap['fighter_sub_wins']=sub_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])

NameError: name 'done' is not defined

In [8]:
ufcfightscrap['fighter_sub_losses']=sub_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])

In [26]:
ufcfightsshitko=pd.read_csv('ufc_fights_shit_ko_fixed.csv',sep=',')

In [27]:
ufcfightscrap['fighter_ko_wins']=ufcfightsshitko['fighter_ko_wins']
ufcfightscrap['fighter_ko_losses']=ufcfightsshitko['fighter_ko_losses']
ufcfightscrap['opponent_ko_wins']=opponent_column('fighter_ko_wins')
ufcfightscrap['opponent_ko_losses']=opponent_column('fighter_ko_losses')
ufcfightscrap['opponent_sub_wins']=opponent_column('fighter_sub_wins')
ufcfightscrap['opponent_sub_losses']=opponent_column('fighter_sub_losses')

In [29]:
#we worked hard to build this, lets save it
ufcfightscrap.to_csv('ufc_fights_crap.csv', index = False)

In [30]:
#list containing all columns of any interest
relevant_list=['date','division','fighter','opponent','result','method']
relevant_list.extend(good_statistics)

In [31]:
#creates a clean file with only columns which are relevant to predicting
ufc_fights=ufcfightscrap[relevant_list]

In [32]:
#lets randomly remove one of every two copied fights
random_indices=[]
for i in range(0,len(ufc_fights['fighter_wins']),2):
    random_indices.append(random.choice([i,i+1]))
    
ufc_fights=ufc_fights.drop(random_indices)

In [33]:
#we worked hard to build this, lets save it
ufc_fights.to_csv('ufc_fights.csv', index = False)